<a href="https://colab.research.google.com/github/RakibRayans/PhD_Candidate_assessment/blob/main/PhD_candidate_assessment_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install pydriller

In [1]:
pip install pydriller pandas matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.9 MB/s eta 0:00:00


In [3]:
!sudo apt-get update
!sudo apt-get upgrade git -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,799 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,063 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,751 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,253 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages 

In [16]:
!pip uninstall pydriller -y
!pip install pydriller

Found existing installation: PyDriller 2.7
Uninstalling PyDriller-2.7:
  Successfully uninstalled PyDriller-2.7
  Using cached PyDriller-2.7-py3-none-any.whl.metadata (1.3 kB)
Using cached PyDriller-2.7-py3-none-any.whl (36 kB)


In [27]:

import matplotlib
matplotlib.use('Agg')
from pydriller import Repository
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Step 1: Extract commit data
repo_url = 'https://github.com/numpy/numpy'
commits = []

print("Extracting data, please wait...")

for commit in Repository(repo_url).traverse_commits():
    commits.append({
        'date': commit.committer_date,
        'author': commit.author.name
    })

# Step 2: Create DataFrame
df = pd.DataFrame(commits)
df['date'] = pd.to_datetime(df['date'], utc=True)
df['month'] = df['date'].dt.to_period('M')  # monthly
df['week'] = df['date'].dt.to_period('W')   # weekly

# Step 3: Commit frequency - Monthly
monthly_commits = df.groupby('month').size().reset_index(name='commits')
monthly_commits['month'] = monthly_commits['month'].astype(str) # Convert month to string for plotting

# Step 4: Plotting
plt.figure(figsize=(14,6))
sns.lineplot(data=monthly_commits, x='month', y='commits', marker='o')
plt.title('Monthly Commit Activity in numpy/numpy')
plt.xlabel('Month')
plt.ylabel('Number of Commits')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("monthly_commit_trend.png")
plt.show()

# Step 5 (Optional): Save data
df.to_csv("numpy_commit_data.csv", index=False)

Extracting data, please wait...


/tmp/ipython-input-27-1219554058.py:26: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = df['date'].dt.to_period('M')  # monthly
/tmp/ipython-input-27-1219554058.py:27: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['week'] = df['date'].dt.to_period('W')   # weekly


In [29]:
from pydriller import Repository
import pandas as pd

repo_url = 'https://github.com/numpy/numpy'
commits = []

for commit in Repository(repo_url).traverse_commits():
    commits.append({
        'hash': commit.hash,
        'author': commit.author.name,
        'author_email': commit.author.email,
        'date': commit.committer_date,
        'message': commit.msg,
        'files': [mod.filename for mod in commit.modified_files],
        'insertions': commit.insertions,
        'deletions': commit.deletions,
        'lines_changed': commit.lines
    })

# Convert to DataFrame
df = pd.DataFrame(commits)

# Optional: Save as CSV
df.to_csv('numpy_commit_data_extended.csv', index=False)

print("CSV saved with extended commit data.")

CSV saved with extended commit data.


In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Load the saved CSV and ensure the 'date' column is in datetime format with UTC
df = pd.read_csv('numpy_commit_data_extended.csv')

# Ensure the 'date' column is in datetime format, coercing errors and converting to UTC
df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True)

# Remove rows where date parsing failed
df.dropna(subset=['date'], inplace=True)

# Step 2: Convert to monthly period
df['month'] = df['date'].dt.to_period('M')

# Step 3: Count number of commits per month
monthly_commit_counts = df.groupby('month').size().reset_index(name='commits')

# Convert the 'month' Period to datetime objects for plotting
monthly_commit_counts['month'] = monthly_commit_counts['month'].dt.to_timestamp()


# Step 4: Plotting the commit trend
plt.figure(figsize=(14, 6))
sns.lineplot(data=monthly_commit_counts, x='month', y='commits', marker='o')
plt.title('Monthly Commit Activity in numpy/numpy', fontsize=14)
plt.xlabel('Month')
plt.ylabel('Number of Commits')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("monthly_commit_trend_Extended.png")
plt.show()

/tmp/ipython-input-33-2891303787.py:15: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = df['date'].dt.to_period('M')
